## Problem Statetment:

![](./data/ProblemStatement.jpg)

## Solution

+ Leveraging a modified version of the Djikstra's shortest path algorithm to solve for this to accomodate for the Grid positions and the finding the adjacent cells in a matrix. 
+ The methodology is described in the methodology tab of this excel sheet. [ShortestPath](./data/ShortestPath.xslx).

In [1]:
import pandas as pd
import numpy as np
import itertools as it

In [2]:
#Function to get the adjacent node co-ordinates from the grid given the current node an the grid shape
def get_adjacent_node_coord(tpl_curr_node, tpl_grid_shape):
    ls_adjacent_node_coords = []
    
    if tpl_curr_node[0] + 1 < tpl_grid_shape[0]:
        ls_adjacent_node_coords.append((tpl_curr_node[0] + 1, tpl_curr_node[1]))
        
    if tpl_curr_node[1] + 1 < tpl_grid_shape[1]:
        ls_adjacent_node_coords.append((tpl_curr_node[0], tpl_curr_node[1] + 1)) 
        
    if (tpl_curr_node[0] + 1 < tpl_grid_shape[0]) and (tpl_curr_node[1] + 1 < tpl_grid_shape[1]):
        ls_adjacent_node_coords.append((tpl_curr_node[0] + 1, tpl_curr_node[1] + 1))
        
    return ls_adjacent_node_coords
        
def ShortestPath_DjikstraMod(ls_grid):
    #Initializing all the values and vectors
    print '*' * 25    
    print 'Create Grid Layouts'
    df_grid = pd.DataFrame(ls_grid)
#     print df_grid

    #Shape of the grids
    d_num_nodes = df_grid.shape[0] * df_grid.shape[1]
    ls_xrange = range(0,df_grid.shape[0])
    ls_yrange = range(0,df_grid.shape[1])

    print 'Create the Node Layout'
    df_nodes = pd.DataFrame(np.array(range(0,d_num_nodes)).reshape(df_grid.shape[0],df_grid.shape[1]))
#     print df_nodes

    print 'Create Node to Grid Coordinates'
    dict_node_to_coordinates = dict(zip(range(0,d_num_nodes),list(it.product(ls_xrange, ls_yrange))))
#     print dict_node_to_coordinates

    print 'Create Grid Coordinates to Node'
    dict_coordinates_to_node = dict(zip(list(it.product(ls_xrange, ls_yrange)),range(0,d_num_nodes)))
#     print dict_coordinates_to_node

    print 'Initializing the Results Matrix'
    df_out = pd.DataFrame(index=range(0,d_num_nodes),columns=['distance','prevnode'])
    #Set the initial distance to zero for the source node
    print 'Setting Source & Node Distances'
    d_source_node = 0
    df_out.loc[d_source_node,'distance'] = 0
    df_out.fillna(float("inf"), inplace=True)
#     print df_out

    d_edge_weight = 1
    print 'Set the Unit Edge Weight', d_edge_weight

    ls_visited_nodes = []
    ls_unvisited_nodes = range(0,d_num_nodes)
    print 'Create list of Unvisited Nodes'
#     print ls_unvisited_nodes

    print 'Calculate Shortest path across Grid/Nodes'
    print 'Using modified Djikstra'
    for node in ls_unvisited_nodes:    
        #Move Node to Visited Node List
        ls_visited_nodes.append(node)    

        #Get Node Co-ordinate and find value at the Co-ordinate
        node_x_coord = dict_node_to_coordinates[node][0]
        node_y_coord = dict_node_to_coordinates[node][1]

        #If the value at the co-ordinate is 'X', then the node is blocked
        if df_grid.iloc[node_x_coord, node_y_coord] == 'X':
            continue

#         print 'Current Node Coordinates:', dict_node_to_coordinates[node], df_grid.iloc[node_x_coord, node_y_coord]

        #If the value at the co-ordinate is 'X', then the node is the objective
        if df_grid.iloc[node_x_coord, node_y_coord] == 'O':
            break        

        #Get Adjacent Nodes using the x, y co-ordinates of the current node
        ls_adjacent_node_coords = get_adjacent_node_coord(dict_node_to_coordinates[node], df_grid.shape)
#         print 'Adjacent Node Coordinates:', ls_adjacent_node_coords

#         For each adjacent node neighbor
#         If neighbor is not blocked i.e. is not 'X'
#             Calculate the distance of the neighbor from the SOURCE Node
#             If the distance from the source node is < the known distance
#                 Update the distance to calculated distance
#                 Update the previous node to the current node

        for adj_node_coords in ls_adjacent_node_coords:
            adj_node = dict_coordinates_to_node[adj_node_coords]

            if df_grid.iloc[adj_node_coords[0],adj_node_coords[1]] == 'X':
                continue

            if (df_out.iloc[node]['distance'] + d_edge_weight) < df_out.iloc[adj_node]['distance']:
                df_out.loc[adj_node,'distance'] = (df_out.iloc[node]['distance'] + d_edge_weight)
                df_out.loc[adj_node,'prevnode'] = node
    print '*'*25
    return df_out, df_nodes, df_grid

## Test Grid: 1

Given the following test Grid, the cells in the grid can be thought of as nodes. So

+ A 3 X 3 grid can be thought of as having a 8 nodes.

In [3]:
#Testing with Grid Layout 1
ls_grid =[['S',None,None],
          [None,'X',None],
          [None,None,'O']]

df_out = pd.DataFrame(ls_grid)
df_nodes = pd.DataFrame(np.array(range(0,df_out.shape[0]*df_out.shape[1])).reshape(df_out.shape[0],df_out.shape[1]))
df_out

,0,1,2
0,S,None,None
1,None,X,None
2,None,None,O


In [4]:
df_nodes

,0,1,2
0,0,1,2
1,3,4,5
2,6,7,8


+ The objective 'O' is at node 8 and source 'S' is at node 0.

In [5]:
df_out, df_nodes, df_grid = ShortestPath_DjikstraMod(ls_grid)

*************************
Create Grid Layouts
Create the Node Layout
Create Node to Grid Coordinates
Create Grid Coordinates to Node
Initializing the Results Matrix
Setting Source & Node Distances
Set the Unit Edge Weight 1
Create list of Unvisited Nodes
Calculate Shortest path across Grid/Nodes
Using modified Djikstra
*************************


In [6]:
df_out

,distance,prevnode
0,0.000000,inf
1,1.000000,0.000000
2,2.000000,1.000000
3,1.000000,0.000000
4,inf,inf
5,2.000000,1.000000
6,2.000000,3.000000
7,2.000000,3.000000
8,3.000000,5.000000


The Above output can be read as follows.
+ To get to the objective 'O' at node 8, we can see that the minimum distance that needs to be travelled is 3. 
+ The minimum route that needs to be followed can be seen from 'prevnode' column.
    + So to reach 'O' at node 8 from 'S' at node 0, we need to traverse through the following nodes 8 <= 5 <= 1 <= 0

## Test Grid: 2

Let's take another examples. Given the below test Grid, the cells in the grid can be thought of as nodes. So

+ A 4 X 4 grid can be thought of as having a 16 nodes.

In [7]:
#Testing with Grid Layout 2
ls_grid =[['S',None,None,None],
          [None,'X',None,None],
          ['X',None,None,None],
          [None,None,None,'O']]

df_out = pd.DataFrame(ls_grid)
df_nodes = pd.DataFrame(np.array(range(0,df_out.shape[0]*df_out.shape[1])).reshape(df_out.shape[0],df_out.shape[1]))
df_out

,0,1,2,3
0,S,None,None,None
1,None,X,None,None
2,X,None,None,None
3,None,None,None,O


In [8]:
df_nodes

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [9]:
df_out, df_nodes, df_grid = ShortestPath_DjikstraMod(ls_grid)

*************************
Create Grid Layouts
Create the Node Layout
Create Node to Grid Coordinates
Create Grid Coordinates to Node
Initializing the Results Matrix
Setting Source & Node Distances
Set the Unit Edge Weight 1
Create list of Unvisited Nodes
Calculate Shortest path across Grid/Nodes
Using modified Djikstra
*************************


In [10]:
df_out

,distance,prevnode
0,0.000000,inf
1,1.000000,0.000000
2,2.000000,1.000000
3,3.000000,2.000000
4,1.000000,0.000000
5,inf,inf
6,2.000000,1.000000
7,3.000000,2.000000
8,inf,inf
9,2.000000,4.000000


The Above output can be read as follows.
+ To get to the objective 'O' at node 15, we can see that the minimum distance that needs to be travelled is 10.
+ The minimum route that needs to be followed can be seen from 'prevnode' column.
    + So to reach 'O' at node 15 from 'S' at node 0, we need to traverse through the following nodes 15 <= 10 <= 6 <= 1 <= 0